In [1]:
from pyomo.environ import *

In [2]:
p1 = 0.88
p2 = 0.82
p3 = 0.92
p4 = 0.84
p5 = 0.73
p6 = 0.87
p7 = 2700.0
p8 = 2300.0
p9 = 540.0
ps = 700000.0

### Non-linear model

In [3]:
model = ConcreteModel(name="Harvesting")

model.f = Var(initialize = 20.0, within=NonNegativeReals)
model.d = Var(initialize = 20.0, within=NonNegativeReals)
model.b = Var(initialize = 20.0, within=NonNegativeReals)
model.hf = Var(initialize = 20.0, within=NonNegativeReals)
model.hd = Var(initialize = 20.0, within=NonNegativeReals)
model.hb = Var(initialize = 20.0, within=NonNegativeReals)
model.br = Var(initialize=1.5, within=NonNegativeReals)
model.c = Var(initialize=500000.0, within=NonNegativeReals)

def obj_rule(m):
    return 10*m.hb + m.hd + m.hf

model.obj = Objective(rule=obj_rule, sense=maximize)

def f_bal_rule(m):
    return m.f == p1*m.br *(p2/10.0*m.f + p3*m.d) - m.hf

model.f_bal = Constraint(rule=f_bal_rule)

def d_bal_rule(m):
    return m.d == p4*m.d + p5/2.0*m.f - m.hd

model.d_bal = Constraint(rule=d_bal_rule)

def b_bal_rule(m):
    return m.b == p6*m.b + p5/2.0*m.f - m.hb

model.b_bal = Constraint(rule=b_bal_rule)

def food_cons_rule(m):
    return m.c == p7*m.b + p8*m.d + p9*m.f

model.food_cons = Constraint(rule=food_cons_rule)

def supply_rule(m):
    return m.c <= ps

model.supply = Constraint(rule=supply_rule)

def birth_rule(m):
    return m.br == 1.1 + 0.8*(ps - m.c)/ps

model.birth = Constraint(rule=birth_rule)

def minbuck_rule(m):
    return m.b >= 1.0/5.0*(0.4*m.f + m.d)

model.minbuck = Constraint(rule=minbuck_rule)

In [4]:
solver = SolverFactory("ipopt")
results = solver.solve(model)
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 8
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.040669918060302734
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [5]:
model.pprint()

8 Var Declarations
    b : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 54.36972761241992 :  None : False : False : NonNegativeReals
    br : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 1.0999999920111345 :  None : False : False : NonNegativeReals
    c : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 700000.0069902573 :  None : False : False : NonNegativeReals
    d : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 196.00640104188517 :  None : False : False : NonNegativeReals
    f : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 189.60559266738446 :  None : False : False : NonNegativeReals
    hb : Size=1, Index=None
        Key  : Lower 

### Linearized model

In [6]:
br = 1.1

In [7]:
model = ConcreteModel(name="Harvesting linearized")

model.f = Var(within=NonNegativeReals)
model.d = Var(within=NonNegativeReals)
model.b = Var(within=NonNegativeReals)
model.hf = Var(within=NonNegativeReals)
model.hd = Var(within=NonNegativeReals)
model.hb = Var(within=NonNegativeReals)
model.c = Var(within=NonNegativeReals)

def obj_rule(m):
    return 10*m.hb + m.hd + m.hf

model.obj = Objective(rule=obj_rule, sense=maximize)

def f_bal_rule(m):
    return m.f == p1*br *(p2/10.0*m.f + p3*m.d) - m.hf

model.f_bal = Constraint(rule=f_bal_rule)

def d_bal_rule(m):
    return m.d == p4*m.d + p5/2.0*m.f - m.hd

model.d_bal = Constraint(rule=d_bal_rule)

def b_bal_rule(m):
    return m.b == p6*m.b + p5/2.0*m.f - m.hb

model.b_bal = Constraint(rule=b_bal_rule)

def food_cons_rule(m):
    return m.c == p7*m.b + p8*m.d + p9*m.f

model.food_cons = Constraint(rule=food_cons_rule)

def supply_rule(m):
    return m.c == ps

model.supply = Constraint(rule=supply_rule)

def minbuck_rule(m):
    return m.b >= 1.0/5.0*(0.4*m.f + m.d)

model.minbuck = Constraint(rule=minbuck_rule)

In [8]:
solver = SolverFactory("glpk")
results = solver.solve(model)
print(results)


Problem: 
- Name: unknown
  Lower bound: 659.224782631462
  Upper bound: 659.224782631462
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 8
  Number of nonzeros: 18
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.06794476509094238
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [9]:
model.pprint()

7 Var Declarations
    b : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 54.3697271084899 :  None : False : False : NonNegativeReals
    c : Size=1, Index=None
        Key  : Lower : Value    : Upper : Fixed : Stale : Domain
        None :     0 : 700000.0 :  None : False : False : NonNegativeReals
    d : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 196.006398762916 :  None : False : False : NonNegativeReals
    f : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 189.605591948833 :  None : False : False : NonNegativeReals
    hb : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 62.1379765372205 :  None : False : False : NonNegativeReals
    hd : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fix